In [1]:
import warnings
warnings.filterwarnings('ignore')

## Bike Sharing Demand
Задача на kaggle: https://www.kaggle.com/c/bike-sharing-demand

По историческим данным о прокате велосипедов и погодным условиям необходимо оценить спрос на прокат велосипедов.

В исходной постановке задачи доступно 11 признаков: https://www.kaggle.com/c/prudential-life-insurance-assessment/data

In [2]:
from sklearn import model_selection, linear_model, metrics

import numpy as np
import pandas as pd

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
raw_data = pd.read_csv('bike_sharing_demand.csv', header = 0, sep = ',')

FileNotFoundError: [Errno 2] File bike_sharing_demand.csv does not exist: 'bike_sharing_demand.csv'

In [ ]:
raw_data.head()

***datetime*** - hourly date + timestamp  

***season*** -  1 = spring, 2 = summer, 3 = fall, 4 = winter 

***holiday*** - whether the day is considered a holiday

***workingday*** - whether the day is neither a weekend nor holiday

***weather*** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
    
***temp*** - temperature in Celsius

***atemp*** - "feels like" temperature in Celsius

***humidity*** - relative humidity

***windspeed*** - wind speed

***casual*** - number of non-registered user rentals initiated

***registered*** - number of registered user rentals initiated

***count*** - number of total rentals

In [ ]:
print(raw_data.shape)

In [ ]:
raw_data.isnull().values.any()

### Предобработка данных

#### Типы признаков

In [ ]:
raw_data.info()

в первом столбце у нас записано дата. 
преобразуем этот тип в datetime.

In [ ]:
raw_data.datetime = raw_data.datetime.apply(pd.to_datetime)

на основе этого столбца рассчитаем два новых признако. 
месяц и час.

In [ ]:
raw_data['month'] = raw_data.datetime.apply(lambda x : x.month)
raw_data['hour'] = raw_data.datetime.apply(lambda x : x.hour)

In [ ]:
raw_data.head()

наши данные имеют явную временную привязку.

Построим модель на данных за более ранний период 
и оценим качество на данных за более поздний период.

#### Обучение и отложенный тест

In [ ]:
train_data = raw_data.iloc[:-1000, :]
hold_out_test_data = raw_data.iloc[-1000:, :]

In [ ]:
print(raw_data.shape, train_data.shape, hold_out_test_data.shape)

In [ ]:
print('train period from {} to {}'.format(train_data.datetime.min(), train_data.datetime.max()))
print('evaluation period from {} to {}'.format(hold_out_test_data.datetime.min(), hold_out_test_data.datetime.max()))

#### Данные и целевая функция

In [ ]:
#обучение
train_labels = train_data['count'].values
train_data = train_data.drop(['datetime', 'count'], axis = 1)

In [ ]:
#тест
test_labels = hold_out_test_data['count'].values
test_data = hold_out_test_data.drop(['datetime', 'count'], axis = 1)

#### Целевая функция на обучающей выборке и на отложенном тесте

In [ ]:
pylab.figure(figsize = (16, 6))

pylab.subplot(1,2,1)
pylab.hist(train_labels)
pylab.title('train data')

pylab.subplot(1,2,2)
pylab.hist(test_labels)
pylab.title('test data')

начнем с числовых признаков.

In [ ]:
numeric_columns = ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'month', 'hour']

In [ ]:
train_data = train_data[numeric_columns]
test_data = test_data[numeric_columns]

In [ ]:
train_data.head()

In [ ]:
test_data.head()

### Модель

In [ ]:
regressor = linear_model.SGDRegressor(random_state = 0, max_iter=5)
regressor.get_params()

In [ ]:
regressor.fit(train_data, train_labels)
metrics.mean_absolute_error(test_labels, regressor.predict(test_data))

получили какую-то невероятно большую ошибку.

In [ ]:
print(test_labels[:10])

In [ ]:
print(regressor.predict(test_data)[:10])

In [ ]:
regressor.coef_

видно что у нас невероятно большие коэффициенты.

Мы работаем с набором данных, в котором признаки сильно отличаются по масштабу.

отмасштабируем признаки в нашем наборе данных.

### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

для того чтобы не использовать информацию из будущего обучим наш scaler на тренировочных данных.

In [ ]:
#создаем стандартный scaler
scaler = StandardScaler()
scaler.fit(train_data, train_labels)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
regressor.fit(scaled_train_data, train_labels)
metrics.mean_absolute_error(test_labels, regressor.predict(scaled_test_data))

ошибка стала очень маленькой.

In [ ]:
print(test_labels[:10])

In [ ]:
print(regressor.predict(scaled_test_data)[:10])

### Подозрительно хорошо?

In [ ]:
print(regressor.coef_)

In [ ]:
print(list(map(lambda x : round(x, 2), regressor.coef_)))

практически все признаки принимают очень маленькие веса за исключением двух.

In [ ]:
train_data.head()

In [ ]:
train_labels[:10]

In [ ]:
np.all(train_data.registered + train_data.casual == train_labels)

фактический два этих столбца в сумме дают нашу целевую метку.

вырежем эти данные из нашего набора данных.

In [ ]:
train_data.drop(['casual', 'registered'], axis = 1, inplace = True)
test_data.drop(['casual', 'registered'], axis = 1, inplace = True)

In [ ]:
scaler.fit(train_data, train_labels)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
regressor.fit(scaled_train_data, train_labels)
metrics.mean_absolute_error(test_labels, regressor.predict(scaled_test_data))

выйдем что наша ошибка сильно выросла.

In [ ]:
print(list(map(lambda x : round(x, 2), regressor.coef_)))

мы получили некоторые Baseline. попытаемся его улучшить.

### Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
#создаем pipeline из двух шагов: scaling и классификация
pipeline = Pipeline(steps = [('scaling', scaler), ('regression', regressor)])

In [ ]:
pipeline.fit(train_data, train_labels)
metrics.mean_absolute_error(test_labels, pipeline.predict(test_data))

### Подбор параметров

In [ ]:
pipeline.get_params().keys()

In [ ]:
parameters_grid = {
    'regression__loss' : ['huber', 'epsilon_insensitive', 'squared_loss', ],
    'regression__max_iter' : [3, 5, 10, 50], 
    'regression__penalty' : ['l1', 'l2', 'none'],
    'regression__alpha' : [0.0001, 0.01],
    'scaling__with_mean' : [0., 0.5],
}

In [ ]:
grid_cv = model_selection.GridSearchCV(pipeline, parameters_grid, scoring = 'neg_mean_absolute_error', cv = 4)

In [ ]:
%%time
grid_cv.fit(train_data, train_labels)

In [ ]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

### Оценка по отложенному тесту

In [ ]:
metrics.mean_absolute_error(test_labels, grid_cv.best_estimator_.predict(test_data))

In [ ]:
np.mean(test_labels)

In [ ]:
test_predictions = grid_cv.best_estimator_.predict(test_data)

In [ ]:
print(test_labels[:10])

In [ ]:
print(test_predictions[:10])

фактический наша оптимизация с помощью подбора параметров не помогла нам улучшить модель.

In [ ]:
pylab.figure(figsize=(16, 6))

pylab.subplot(1,2,1)
pylab.grid(True)
pylab.scatter(train_labels, pipeline.predict(train_data), alpha=0.5, color = 'red')
pylab.scatter(test_labels, pipeline.predict(test_data), alpha=0.5, color = 'blue')
pylab.title('no parameters setting')
pylab.xlim(-100,1100)
pylab.ylim(-100,1100)

pylab.subplot(1,2,2)
pylab.grid(True)
pylab.scatter(train_labels, grid_cv.best_estimator_.predict(train_data), alpha=0.5, color = 'red')
pylab.scatter(test_labels, grid_cv.best_estimator_.predict(test_data), alpha=0.5, color = 'blue')
pylab.title('grid search')
pylab.xlim(-100,1100)
pylab.ylim(-100,1100)

обработаем все признаки.

разделим наши данные по типам.

In [ ]:
from sklearn import pipeline, preprocessing

In [ ]:
raw_data.head()

In [ ]:
train_data = raw_data.iloc[:-1000, :]
hold_out_test_data = raw_data.iloc[-1000:, :]

In [ ]:
print(raw_data.shape, train_data.shape, hold_out_test_data.shape)

In [ ]:
#обучение
train_labels = train_data['count'].values
train_data = train_data.drop(['datetime', 'count', 'casual', 'registered'], axis = 1)

In [ ]:
#тест
test_labels = hold_out_test_data['count'].values
test_data = hold_out_test_data.drop(['datetime', 'count', 'casual', 'registered'], axis = 1)

получим логический индекс который показывает в каком столбце находятся данные данного типа.

In [ ]:
binary_data_columns = ['holiday', 'workingday']
binary_data_indices = np.array([(column in binary_data_columns) for column in train_data.columns], dtype = bool)

In [ ]:
print(binary_data_columns)
print(binary_data_indices)

In [ ]:
categorical_data_columns = ['season', 'weather', 'month'] 
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)

In [ ]:
print(categorical_data_columns)
print(categorical_data_indices)

In [ ]:
numeric_data_columns = ['temp', 'atemp', 'humidity', 'windspeed', 'hour']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)

In [ ]:
print(numeric_data_columns)
print(numeric_data_indices)

Pipeline

построим цепочку преобразованией в результате которых получим dataset состоящий из обработанных данных.

In [ ]:
regressor = linear_model.SGDRegressor(random_state = 0, n_iter = 3, loss = 'squared_loss', penalty = 'l2')

In [ ]:
estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices])),
                ('scaling', preprocessing.StandardScaler(with_mean = 0))            
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ])),
    ('model_fitting', regressor)
    ]
)

In [ ]:
estimator.fit(train_data, train_labels)

In [ ]:
metrics.mean_absolute_error(test_labels, estimator.predict(test_data))

получили примерно тоже самое, фактический преобразование не помогло принципиально улучшить модели.

попробуем на это влиять подбором параметров.

In [ ]:
estimator.get_params().keys()

переберем два параметра.

In [ ]:
parameters_grid = {
    'model_fitting__alpha' : [0.0001, 0.001, 0,1],
    'model_fitting__eta0' : [0.001, 0.05],
}

In [ ]:
grid_cv = model_selection.GridSearchCV(estimator, parameters_grid, scoring = 'neg_mean_absolute_error', cv = 4)

In [ ]:
%%time
grid_cv.fit(train_data, train_labels)

In [ ]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

оценим качество по отложенному тесту.

In [ ]:
test_predictions = grid_cv.best_estimator_.predict(test_data)

In [ ]:
metrics.mean_absolute_error(test_labels, test_predictions)

получаем не очень хороший результат.

In [ ]:
print(test_labels[:20])

In [ ]:
print(test_predictions[:20])

отобрази наши объекты в координатах исходных значений целевой метки и наших прогнозов.

In [ ]:
pylab.figure(figsize=(8, 6))
pylab.grid(True)
pylab.xlim(-100,1100)
pylab.ylim(-100,1100)
pylab.scatter(train_labels, grid_cv.best_estimator_.predict(train_data), alpha=0.5, color = 'red')
pylab.scatter(test_labels, grid_cv.best_estimator_.predict(test_data), alpha=0.5, color = 'blue')

все наши преобразования не перевели к улучшению модели.

Мы строили линейную модель, это говорит о том, что мы предполагаем некоторую линейную зависимость
между признаками и целевой переменной.

Построим нелинейную модель, которая умеет учитывать нелинейные зависимости между признаками и целевой функцией.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regressor = RandomForestRegressor(random_state = 0, max_depth = 20, n_estimators = 50)

In [ ]:
estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices])),
                ('scaling', preprocessing.StandardScaler(with_mean = 0))            
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ])),
    ('model_fitting', regressor)
    ]
)

In [ ]:
estimator.fit(train_data, train_labels)

In [ ]:
metrics.mean_absolute_error(test_labels, estimator.predict(test_data))

ошибка сильно уменьшилась.

In [ ]:
test_labels[:10]

In [ ]:
estimator.predict(test_data)[:10]

сравним график линейной модели и случайного леса.

In [ ]:
pylab.figure(figsize=(16, 6))

pylab.subplot(1,2,1)
pylab.grid(True)
pylab.xlim(-100,1100)
pylab.ylim(-100,1100)
pylab.scatter(train_labels, grid_cv.best_estimator_.predict(train_data), alpha=0.5, color = 'red')
pylab.scatter(test_labels, grid_cv.best_estimator_.predict(test_data), alpha=0.5, color = 'blue')
pylab.title('linear model')

pylab.subplot(1,2,2)
pylab.grid(True)
pylab.xlim(-100,1100)
pylab.ylim(-100,1100)
pylab.scatter(train_labels, estimator.predict(train_data), alpha=0.5, color = 'red')
pylab.scatter(test_labels, estimator.predict(test_data), alpha=0.5, color = 'blue')
pylab.title('random forest model')

в данном случае наши объекты ты достаточно близко подошли к диагональной области.

с помощью случайного леса, получилось установить зависимость гораздо лучше.